# VacationPy

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
import api_keys as keys

In [36]:
df_city_weather = pd.read_csv(os.path.join('..','Resources', 'city_weather.csv')).sample(500)
df_ideal_weather = df_city_weather.loc[(df_city_weather.temp_max >= 65)
                                     & (df_city_weather.temp_max <= 82)
                                     & (df_city_weather.wind < 5)
                                     & (df_city_weather.clouds < 10)]
df_ideal_weather

,country,city,latitude,longitude,hemisphere,temp,humidity,clouds,wind,temp_max
854,MU,grand gaube,-20.006389,57.660833,southern,68.00,67.0,0.0,3.00,68.00
377,ES,baeza,37.993839,-3.471029,northern,76.98,35.0,0.0,3.00,79.00
331,GR,rizarion,40.787222,22.083889,northern,70.14,73.0,1.0,0.89,72.00
280,IN,karanpura,29.841944,73.454167,northern,73.33,18.0,0.0,4.07,73.33
379,GR,kato soulion,38.166667,24.000000,northern,62.11,56.0,0.0,1.99,66.99
290,IN,aligarh,27.883333,78.083333,northern,72.28,21.0,0.0,2.39,72.28
252,IN,gagret,31.666667,76.066667,northern,67.06,26.0,0.0,2.91,67.06
1064,AR,puerto madryn,-42.769201,-65.038513,southern,79.54,17.0,0.0,4.12,79.54
661,EC,cariamanga,-4.333333,-79.550000,southern,78.80,34.0,0.0,2.24,78.80
185,MX,el limon,19.816667,-104.183333,northern,70.72,77.0,0.0,1.32,70.72


In [60]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df = pd.DataFrame(columns=['hotel_name', 'city', 'country', 'lat', 'lng'])

for index, row in df_ideal_weather.iterrows():
    params = {
        "location": f"{row['latitude']}, {row['longitude']}",
        "keyword": "hotel",
        "radius": 5000,
        "type": 'lodging',
        "key": keys.g_key
    }

    response = requests.get(base_url, params=params).json()

    for h in response['results']:
        if h['business_status'] == 'OPERATIONAL':
            hotel_df = hotel_df.append(pd.DataFrame({
                'hotel_name':[h['name']],
                'city':[row['city'].title()],
                'country':[row['country']],
                'lat':[h['geometry']['location']['lat']],
                'lng':[h['geometry']['location']['lng']]
            }))
            break

    

info_box_template = """
<p>Name: {hotel_name}</p>
<p>City: {city}</p>
<p>Country: {country}</p>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [61]:
hotel_df.head(20)

,hotel_name,city,country,lat,lng
0,LUX Grand Gaube Resort & Villas,Grand Gaube,MU,-20.002344,57.659766
0,Hotel Puerta de la Luna,Baeza,ES,37.989860,-3.469757
0,Varosi 4 Seasons,Rizarion,GR,40.801592,22.053675
0,Lisianthos Villa,Kato Soulion,GR,38.156989,24.059962
0,Hotel Orchid Blu,Aligarh,IN,27.893967,78.087021
0,The Hotel Clark Plaza,Gagret,IN,31.694097,76.081267
0,Dazzler by Wyndham Puerto Madryn,Puerto Madryn,AR,-42.769820,-65.030420
0,HOSTAL JV,Cariamanga,EC,-4.329812,-79.556916
0,Hotel Del Camino Real,El Limon,MX,19.804912,-104.217854
0,Jahangardi Hotel,Fasa,IR,28.943129,53.639358


In [63]:
gmaps.configure(api_key=keys.g_key)
heat_locations = df_city_weather[["latitude", "longitude"]].astype(float)
humidity = df_city_weather["humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 1)

marker_locations = hotel_df[['lat', 'lng']]
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))